In [69]:
import pandas as pd
from pathlib import Path

# 1. 현장데이터 폴더 경로 지정
base_path = Path(r"C:\Users\qndba\peer\현장데이터")

# 2. 'wave'로 시작하고 .xlsx 확장자인 파일 모두 찾기
excel_files = list(base_path.rglob("*wave*.xlsx"))

# 3. 데이터프레임을 담을 리스트
dfs = []

# 4. 각 엑셀에서 '설문지 응답 시트1' 시트를 읽어오기
for file in excel_files:
    try:
        print(file)
        df = pd.read_excel(file, sheet_name='설문지 응답 시트1', engine='openpyxl')
        df['source_file'] = file.name  # 출처 파일명 기록
        dfs.append(df)
    except Exception as e:
        print(f"파일 {file.name} 에서 오류 발생: {e}")

C:\Users\qndba\peer\현장데이터\2024 목포부주초 Wave1_6-5(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_4-1 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_4-3 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_4-4 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_5-2 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_5-4(학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_5-5 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 송수초 Wave1_6-5 (학생)(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_4-1(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_4-2(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_4-3(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_5-1(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_5-2(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_5-3(응답).xlsx
C:\Users\qndba\peer\현장데이터\2024 하원초 Wave1_5-4(응답).xlsx
C:\U

In [68]:
personinfo_df = df.loc[:,['타임스탬프', '학년', '반', '번호', '성별', '이름']]
keywords = [
    "가장 인기가 있는",
    "가장 친한 친구는?",
    "학급에서 영향력이 있",
    "친구들의 생각을 자주",
    "험담을 하거나 다른 친",
    "나쁜 소문을 퍼뜨리는",
    "친절한",
    "협력적인"
]

# 전체 dfs 처리 결과를 담을 리트트
all_item_matrix_dicts = []

for i, df in enumerate(dfs): # i가 각 학급 순서를 의미함
    filtered_cols = [col for col in df.columns if any(keyword in col for keyword in keywords)]
    filtered_df = df[filtered_cols]
    personinfo_df = df.iloc[:,:6]
    row_df = pd.concat([personinfo_df, filtered_df], ignore_index=False, axis=1)
    
    # 전처리된거에서 인적정보 다시 분리
    rowinfo_df = row_df.iloc[:,:6]
    print(rowinfo_df['이름'].tolist())

    # 아이템 분리 후
    rowitem_df = row_df.iloc[:,6:]
    # 문항별로 또 분리
    keywords_dict = {
        'PN1': "가장 인기가 있는",
        'PN14': "가장 친한 친구는?",
        'PN4': "학급에서 영향력이 있",
        'PN5': "친구들의 생각을 자주",
        'PN6': "험담을 하거나 다른 친",
        'PN7': "나쁜 소문을 퍼뜨리는",
        'PN8': "친절한",
        'PN9': "협력적인"
    }
    # 문항별 데이터프레임을 담을 딕셔너리
    item_matrix_dict = {}
    # keywords_dict에 따라 필터링된 데이터프레임을 저장
    for item_no, keyword in keywords_dict.items():
        temp_df = rowitem_df.filter(regex=keyword)
        temp_df = temp_df.fillna(0)
        labels = rowinfo_df['이름'].tolist()
        temp_df.index = labels
        temp_df.columns = labels
        item_matrix_dict[item_no] = temp_df # 문항 하나에 해당 응답 df 저장
        
    # i의 순서 학급에 문항별 응답 df가 저장된 딕셔너리
    all_item_matrix_dicts.append(item_matrix_dict)

    all_triplets = []

    for item_no, item_df in item_matrix_dict.items():
        idx_pairs = item_df.where(item_df == 1).stack().index.tolist() # 이 코드 공부하기#############
        for r, c in idx_pairs:
            all_triplets.append({'point_item_no': item_no, 'student_designation': r, 'student_nomination': c})
        triplet_df = pd.DataFrame(all_triplets)
    triplet_df.to_csv(f'C:/Users/qndba/peer/현장데이터/class_{i+1}.csv', encoding='utf-8-sig')

['51강*욱', '52곽*중', '53김*재', '54김*은', '55김*우', '56김*홍', '57김*윤', '58박*형', '59박*린', '510박*우', '511박*유', '512배*을', '513서*웅', '514오*서', '516이*영', '517이*', '518*훈', '519장*준', '520장*수', '521정***타', '522조*민', '523진*랑', '524최*원', '525최*예', '526홍*경']
['강민하', '고강민', '김무성', '김성빈', '김성윤', '박지후', '서도현', '손정우', '오은율', '이유준', '이한', '정민준', '정준휘', '한정우', '김가영', '김서영', '김서현', '김소율', '김채아', '김태연', '문하율', '문하음', '서지현', '정세윤', '조윤채', '최소은', '최시아']
['김관우', '김병하', '김태훈', '남윤우', '신원재', '윤대웅', '이범준', '이준서', '장지욱', '정해람', '조예준', '홍종빈', '황규진', '강주은', '고나은', '김나현', '김다혜', '김서하', '김세연', '밀라드아리아로빈은유', '박여울', '변서유', '송유안', '이하린', '정라원', '정은우', '한서연']
['강영재', '김도윤', '김서준', '김서진', '김주원', '김지한', '김하랑', '손호진', '이승유', '이승주', '정재준', '조영원', '홍주현', '조민성', '강민지', '김서현', '김시호', '남유진', '안소율', '양라빈', '유연우', '윤희진', '이수아', '이지우', '조은우', '표윤희', '허은수']
['강윤우', '김정진', '김채우', '김한겸', '박지성', '백동우', '백준우', '이동건', '임지원', '표재민', '홍주원', '강보민', '김라은', '김민서', '김태린', '박지아', '예지우', '오예진', '이서현', '정예슬', '최연우', '하지민', '한예주', '허수빈']
['강우진', '김승윤'